# Import required libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import datetime, timedelta
import uuid
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("🎮 Database-Compatible Game Recommendation System")
print("=" * 60)
print("✅ Libraries imported successfully!")

🎮 Database-Compatible Game Recommendation System
✅ Libraries imported successfully!


# Define enums matching your database

In [ ]:
ROLES = ['USER', 'ADMIN']
GENDERS = ['MALE', 'FEMALE']
CATEGORIES = [
    'Action', 'Adventure', 'RPG', 'Strategy',
    'Sports', 'Racing', 'Puzzle', 'Shooter', 'Simulation'
]

def generate_database_compatible_data():
    """Generate data matching your database schema exactly"""

    # 1. CATEGORY TABLE
    categories_db = []
    for i, cat_name in enumerate(CATEGORIES, 1):
        categories_db.append({
            'id': str(uuid.uuid4()),
            'name': cat_name,
            'createdAt': datetime.now()
        })
    categories_df = pd.DataFrame(categories_db)

    # 2. USERS TABLE
    users_db = []
    base_date = datetime.now()

    for i in range(1, 501):  # 500 users
        # Generate birth date (age 8-65)
        age = np.random.randint(8, 66)
        birth_date = base_date - timedelta(days=age*365 + np.random.randint(0, 365))

        users_db.append({
            'id': str(uuid.uuid4()),
            'name': f'User{i}',
            'username': f'user{i}',
            'email': f'user{i}@example.com',
            'password': 'hashed_password',
            'role': np.random.choice(ROLES, p=[0.95, 0.05]),
            'birthDate': birth_date.date(),
            'gender': np.random.choice(GENDERS),
            'imageUrl': f'profile{i}.jpg',
            'createdAt': base_date.date(),
            'updatedAt': base_date.date()
        })

    users_df = pd.DataFrame(users_db)

    # 3. CONTENTITEM TABLE
    content_items_db = []
    for i in range(1, 1001):  # 1000 games
        content_items_db.append({
            'id': str(uuid.uuid4()),
            'name': f'Game {i}',
            'description': f'Description for Game {i}',
            'imageUrl': f'game{i}.jpg',
            'type': 'GAME',
            'createdAt': (base_date - timedelta(days=np.random.randint(0, 2000)))
        })

    content_items_df = pd.DataFrame(content_items_db)

    # 4. GAME TABLE
    games_db = []
    for i, content_item in enumerate(content_items_df.to_dict('records')):
        category_id = categories_df.iloc[np.random.randint(0, len(categories_df))]['id']
        games_db.append({
            'id': content_item['id'],
            'categoryId': category_id,
            'minAge': np.random.choice([3, 7, 13, 17, 18], p=[0.25, 0.35, 0.25, 0.10, 0.05]),
            'targetGender': assign_target_gender_for_category(category_id, categories_df),
            'averageRating': round(np.random.uniform(2.5, 5.0), 2)
        })

    games_df = pd.DataFrame(games_db)

    print(f"Generated {len(categories_df)} categories")
    print(f"Generated {len(users_df)} users")
    print(f"Generated {len(content_items_df)} content items")
    print(f"Generated {len(games_df)} games")

    return categories_df, users_df, content_items_df, games_df

def assign_target_gender_for_category(category_id, categories_df):
    """Assign target gender based on category"""
    category_name = categories_df[categories_df['id'] == category_id]['name'].iloc[0]

    gender_mapping = {
        'Action': np.random.choice(['MALE', 'MALE', 'FEMALE'], p=[0.7, 0.2, 0.1]),
        'Adventure': np.random.choice(['FEMALE', 'MALE', 'FEMALE'], p=[0.4, 0.3, 0.3]),
        'RPG': np.random.choice(['MALE', 'FEMALE'], p=[0.6, 0.4]),
        'Strategy': np.random.choice(['MALE', 'FEMALE'], p=[0.8, 0.2]),
        'Sports': 'MALE',
        'Racing': np.random.choice(['MALE', 'FEMALE'], p=[0.8, 0.2]),
        'Puzzle': np.random.choice(['FEMALE', 'MALE'], p=[0.7, 0.3]),
        'Shooter': np.random.choice(['MALE', 'FEMALE'], p=[0.9, 0.1]),
        'Simulation': np.random.choice(['MALE', 'FEMALE'], p=[0.6, 0.4])
    }

    return gender_mapping.get(category_name, np.random.choice(['MALE', 'FEMALE']))

def calculate_user_age(birth_date):
    """Calculate age from birthDate"""
    today = datetime.now().date()
    return today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))

# Generate database-compatible data
categories_df, users_df, content_items_df, games_df = generate_database_compatible_data()

# Add calculated age column for easier processing
users_df['age_calculated'] = users_df['birthDate'].apply(calculate_user_age)

print(f"\n Database Schema Summary:")
print(f"Categories: {list(categories_df['name'].unique())}")
print(f"User Genders: {dict(users_df['gender'].value_counts())}")
print(f"User Ages: {users_df['age_calculated'].min()}-{users_df['age_calculated'].max()}")
print(f"Game Age Requirements: {dict(games_df['minAge'].value_counts())}")

✅ Generated 9 categories
✅ Generated 500 users
✅ Generated 1000 content items
✅ Generated 1000 games

📊 Database Schema Summary:
Categories: ['Action', 'Adventure', 'RPG', 'Strategy', 'Sports', 'Racing', 'Puzzle', 'Shooter', 'Simulation']
User Genders: {'MALE': 262, 'FEMALE': 238}
User Ages: 8-65
Game Age Requirements: {7: 356, 13: 259, 3: 255, 17: 80, 18: 50}


# Create users dataset with category preferences

In [ ]:
print("👥 Creating Enhanced Users Dataset...")

users_data = {
    'id': range(1, 501),
    'age_calculated': users_df['age_calculated'].values,
    'gender': users_df['gender'].apply(lambda x: 'Male' if x == 'MALE' else 'Female').values,  # Only Male/Female
    'preferred_category': np.random.choice(categories_df['name'].values, 500),  # Primary category preference
}

# Create simple users dataframe with numeric IDs for easier processing
users_df_simple = pd.DataFrame(users_data)

# Add derived user features
users_df_simple['birthDate'] = users_df['birthDate'].values
users_df_simple['age_group'] = pd.cut(users_df_simple['age_calculated'], 
                                bins=[0, 12, 18, 25, 35, 50, 100],
                                labels=['Child', 'Teen', 'Young_Adult', 'Adult', 'Middle_Age', 'Senior'])

print(" Users Dataset Created")
print(f" Dataset shape: {users_df_simple.shape}")
print(f" Gender distribution: {dict(users_df_simple['gender'].value_counts())}")
print(f" Preferred categories: {dict(users_df_simple['preferred_category'].value_counts())}")

# Display sample
print("\n Sample Users:")
print(users_df_simple.head())

# Save users dataset
users_df_simple.to_csv('enhanced_users_dataset_500.csv', index=False)
print(f"\n Users dataset saved as 'enhanced_users_dataset_500.csv'")

👥 Creating Enhanced Users Dataset...
✅ Users Dataset Created
📊 Dataset shape: (500, 6)
📊 Gender distribution: {'Male': 262, 'Female': 238}
📊 Preferred categories: {'Shooter': 72, 'Sports': 63, 'Strategy': 62, 'Puzzle': 58, 'Simulation': 52, 'Racing': 51, 'Action': 50, 'RPG': 47, 'Adventure': 45}

📋 Sample Users:
   id  age_calculated  gender preferred_category   birthDate    age_group
0   1              26    Male             Action  1998-10-28        Adult
1   2              58  Female          Adventure  1966-11-21       Senior
2   3              25  Female             Puzzle  1999-10-15  Young_Adult
3   4              26  Female                RPG  1998-08-22        Adult
4   5              50    Male             Action  1975-05-20   Middle_Age

✅ Users dataset saved as 'enhanced_users_dataset_500.csv'


# Generate GameRating and UserGame tables with realistic user behavior

In [ ]:
def generate_user_interactions(users_df_simple, games_df, categories_df):
    """Generate GameRating and UserGame tables based on user preferences"""
    print(" Generating GameRating and UserGame tables...")

    game_ratings_db = []
    user_games_db = []
    rating_counter = 1
    user_game_counter = 1

    for _, user in users_df_simple.iterrows():
        user_id = user["id"]  # Use simple numeric ID for easier processing
        user_age = user['age_calculated']
        user_gender = user['gender'].upper()  # Convert 'Male'/'Female' to 'MALE'/'FEMALE'

        # Determine user's preferred categories based on demographics
        preferred_categories = get_user_preferred_categories(user_age, user_gender)

        # Number of games user has interacted with (based on realistic patterns)
        num_games_rated = np.random.randint(5, 25)  # User rates 5-25 games
        num_games_played = np.random.randint(3, 15)  # User plays 3-15 games

        # Select games user can play (age appropriate)
        eligible_games = games_df[games_df['minAge'] <= user_age]

        if len(eligible_games) == 0:
            continue

        # Generate GameRatings
        rated_games = eligible_games.sample(n=min(num_games_rated, len(eligible_games)))

        for _, game in rated_games.iterrows():
            # Get category name for preference calculation
            category_name = categories_df[categories_df['id'] == game['categoryId']]['name'].iloc[0]

            # Calculate rating based on preferences
            base_rating = 3.0  # Neutral starting point

            # Category preference bonus
            if category_name in preferred_categories[:2]:  # Top 2 preferred
                base_rating += np.random.uniform(0.5, 2.0)
            elif category_name in preferred_categories:  # Other preferred
                base_rating += np.random.uniform(0.0, 1.0)
            else:  # Not preferred
                base_rating += np.random.uniform(-1.0, 0.5)

            # Gender targeting bonus
            if game['targetGender'] == user_gender:
                base_rating += np.random.uniform(0.2, 0.8)

            # Game quality influence
            quality_factor = (game['averageRating'] - 2.5) / 2.5  # Normalize to 0-1
            base_rating += quality_factor * np.random.uniform(0.3, 1.0)

            # Add randomness
            base_rating += np.random.uniform(-0.5, 0.5)

            # Ensure rating is in valid range 1-5
            final_rating = max(1, min(5, int(round(base_rating))))

            game_ratings_db.append({
                'id': f'note_{rating_counter:06d}',
                'userId': user_id,
                'gameId': game['id'],
                'rating': final_rating,
                'ratedAt': (datetime.now() - timedelta(days=np.random.randint(1, 365))).date()
            })
            rating_counter += 1

        # Generate UserGames (subset of rated games)
        played_games = rated_games.sample(n=min(num_games_played, len(rated_games)))

        for _, game in played_games.iterrows():
            # Score influenced by user's rating of the game
            user_rating_for_game = next((n['rating'] for n in game_ratings_db
                                     if n['userId'] == user_id and n['gameId'] == game['id']), 3)

            # Higher ratings tend to produce higher scores
            base_score = user_rating_for_game * 200  # Base correlation
            score_variation = np.random.randint(-200, 300)  # Random skill factor
            final_score = max(0, base_score + score_variation)

            user_games_db.append({
                'id': f'score_{user_game_counter:06d}',
                'userId': user_id,
                'gameId': game['id'],
                'score': final_score,
                'playedAt': (datetime.now() - timedelta(days=np.random.randint(1, 180))).date()
            })
            user_game_counter += 1

    game_ratings_df = pd.DataFrame(game_ratings_db)
    user_games_df = pd.DataFrame(user_games_db)

    print(f" Generated {len(game_ratings_df)} game ratings")
    print(f" Generated {len(user_games_df)} user game plays")

    return game_ratings_df, user_games_df

def get_user_preferred_categories(age, gender):
    """Determine user's preferred categories based on demographics"""
    base_preferences = {
        'MALE': {
            'child': ['Action', 'Adventure', 'Racing'],
            'teen': ['Action', 'Shooter', 'Sports', 'Racing'],
            'adult': ['Strategy', 'Shooter', 'Action', 'Simulation'],
            'senior': ['Strategy', 'Puzzle', 'Simulation']
        },
        'FEMALE': {
            'child': ['Adventure', 'Puzzle', 'Simulation'],
            'teen': ['Adventure', 'RPG', 'Puzzle'],
            'adult': ['RPG', 'Adventure', 'Strategy', 'Puzzle'],
            'senior': ['Puzzle', 'Strategy', 'Adventure']
        }
    }

    # Determine age group
    if age <= 12:
        age_group = 'child'
    elif age <= 18:
        age_group = 'teen'
    elif age <= 50:
        age_group = 'adult'
    else:
        age_group = 'senior'

    return base_preferences[gender][age_group]

# Generate interaction tables
game_ratings_df, user_games_df = generate_user_interactions(users_df_simple, games_df, categories_df)

# Display sample data
print(f"\n Sample Database Records:")
print("\nCATEGORIES:")
print(categories_df.head())
print("\nUSERS (sample):")
print(users_df_simple[['id', 'age_calculated', 'gender', 'preferred_category']].head())
print("\nCONTENT ITEMS (sample):")
print(content_items_df.head())
print("\nGAMES (sample):")
print(games_df.head())
print("\nGAME RATINGS (sample):")
print(game_ratings_df.head())
print("\nUSER GAMES (sample):")
print(user_games_df.head())

📝 Generating GameRating and UserGame tables...
✅ Generated 7157 game ratings
✅ Generated 3923 user game plays

📋 Sample Database Records:

CATEGORIES:
                                     id       name                  createdAt
0  e5d5782d-c340-4247-ad2a-2d4afd7416bc     Action 2025-06-01 01:37:18.968753
1  27791a38-0fa3-4311-87c9-09e621f02677  Adventure 2025-06-01 01:37:18.968753
2  0f6dfd32-de57-4241-a71a-8ade7ca395f6        RPG 2025-06-01 01:37:18.968753
3  95d534e8-2fb8-41d0-abc9-37c05d42f43c   Strategy 2025-06-01 01:37:18.968753
4  09d4be46-f2cd-497f-a34e-59037e71629a     Sports 2025-06-01 01:37:18.968753

USERS (sample):
   id  age_calculated  gender preferred_category
0   1              26    Male             Action
1   2              58  Female          Adventure
2   3              25  Female             Puzzle
3   4              26  Female                RPG
4   5              50    Male             Action

CONTENT ITEMS (sample):
                                     id    na

# Create Database-Compatible Feature Engineering

In [ ]:
def create_database_features():
    """Create features using exact database schema"""
    print(" Creating Database-Compatible Features...")

    # Merge all tables to create comprehensive dataset
    # First join games with content_items to get the game names
    games_with_names = games_df.merge(
        content_items_df[['id', 'name']],
        on='id'
    )
    
    # Then join with categories
    games_with_categories = games_with_names.merge(
        categories_df.rename(columns={'name': 'category_name'}),
        left_on='categoryId',
        right_on='id',
        suffixes=('', '_cat')
    )

    # Join game ratings with users and games
    full_dataset = game_ratings_df.merge(
        users_df_simple[['id', 'birthDate', 'gender', 'age_calculated']],
        left_on='userId',
        right_on='id',
        suffixes=('', '_user')
    ).merge(
        games_with_categories[['id', 'name', 'categoryId', 'minAge', 'targetGender', 'averageRating', 'category_name']],
        left_on='gameId',
        right_on='id',
        suffixes=('', '_game')
    )

    # Add score information
    user_scores = user_games_df.groupby('userId').agg({
        'score': ['mean', 'count', 'max'],
        'playedAt': 'max'
    }).round(2)
    user_scores.columns = ['avg_score', 'games_played', 'max_score', 'last_played']
    user_scores = user_scores.reset_index()

    full_dataset = full_dataset.merge(
        user_scores,
        left_on='userId',
        right_on='userId',
        how='left'
    )

    # Fill missing score data
    full_dataset['avg_score'] = full_dataset['avg_score'].fillna(500)
    full_dataset['games_played'] = full_dataset['games_played'].fillna(1)
    full_dataset['max_score'] = full_dataset['max_score'].fillna(500)

    print(f" Created merged dataset: {full_dataset.shape}")

    # Analyze user category preferences from their actual ratings
    user_category_preferences = analyze_database_user_preferences(full_dataset)

    # Create feature matrix
    features_list = []

    print("   Processing user-game combinations...")
    for idx, row in full_dataset.iterrows():
        if idx % 1000 == 0:
            print(f"   Progress: {idx}/{len(full_dataset)}")

        user_id = row['userId']
        user_prefs = user_category_preferences.get(user_id, {})

        # Basic user features
        features = {
            'user_age': row['age_calculated'],
            'user_gender_MALE': 1 if row['gender'] == 'MALE' else 0,
            'user_gender_FEMALE': 1 if row['gender'] == 'FEMALE' else 0,

            # User gaming activity
            'user_avg_score': row['avg_score'],
            'user_games_played': row['games_played'],
            'user_max_score': row['max_score'],

            # Game features
            'game_average_rating': row['averageRating'],
            'game_min_age': row['minAge'],
            'game_target_gender_MALE': 1 if row['targetGender'] == 'MALE' else 0,
            'game_target_gender_FEMALE': 1 if row['targetGender'] == 'FEMALE' else 0,

            # Compatibility features
            'age_appropriate': 1 if row['age_calculated'] >= row['minAge'] else 0,
            'gender_target_match': 1 if row['gender'] == row['targetGender'] else 0,
        }

        # Category features - KEY FOR RECOMMENDATIONS
        current_category = row['category_name']
        for category in CATEGORIES:
            # Binary feature for current game category
            features[f'game_category_{category}'] = 1 if current_category == category else 0

            # User's preference score for this category
            if category in user_prefs:
                features[f'user_pref_{category}'] = user_prefs[category] / 5.0  # Normalize
            else:
                features[f'user_pref_{category}'] = 0.6  # Neutral preference

        # Current game category preference score
        features['category_preference_score'] = user_prefs.get(current_category, 3.0) / 5.0

        # Is this user's most preferred category?
        if user_prefs:
            top_category = max(user_prefs.items(), key=lambda x: x[1])[0]
            features['is_top_preferred_category'] = 1 if current_category == top_category else 0
        else:
            features['is_top_preferred_category'] = 0

        # Interaction features
        features.update({
            'age_quality_interaction': row['age_calculated'] * row['averageRating'],
            'preference_quality_interaction': features['category_preference_score'] * row['averageRating'],
            'activity_preference_interaction': row['games_played'] * features['category_preference_score'],
            'score_preference_interaction': (row['avg_score'] / 1000) * features['category_preference_score']
        })

        features_list.append(features)

    features_df = pd.DataFrame(features_list)
    target = full_dataset['rating']  # The rating to predict

    print(f" Database features created: {features_df.shape}")
    print(f" Features include: user demographics, game properties, category preferences, activity metrics")

    return features_df, target, full_dataset, user_category_preferences

def analyze_database_user_preferences(dataset):
    """Analyze user category preferences from GameRating table"""
    user_preferences = {}

    for user_id in dataset['userId'].unique():
        user_ratings = dataset[dataset['userId'] == user_id]

        # Calculate average rating per category
        category_ratings = user_ratings.groupby('category_name')['rating'].agg(['mean', 'count'])

        # Only consider categories with at least 2 ratings for reliability
        reliable_categories = category_ratings[category_ratings['count'] >= 2]

        if len(reliable_categories) > 0:
            # Weight by both rating and frequency
            preferences = {}
            for category, stats in reliable_categories.iterrows():
                avg_rating = stats['mean']
                count = stats['count']
                # Boost preference score slightly for more played categories
                weighted_score = avg_rating * (1 + np.log(count) * 0.1)
                preferences[category] = weighted_score

            user_preferences[user_id] = preferences

    return user_preferences

# Create database-compatible features
db_features_df, db_target, db_full_dataset, db_user_preferences = create_database_features()

print(f"\n Database Feature Summary:")
print(f"   Total features: {len(db_features_df.columns)}")
print(f"   Key category features: category_preference_score, is_top_preferred_category")
print(f"   Database-specific features: user_avg_score, games_played, game_average_rating")
print(f"   Target variable: rating (1-5 rating scale)")

🔧 Creating Database-Compatible Features...
✅ Created merged dataset: (7157, 20)
   Processing user-game combinations...
   Progress: 0/7157
   Progress: 1000/7157
   Processing user-game combinations...
   Progress: 0/7157
   Progress: 1000/7157
   Progress: 2000/7157
   Progress: 3000/7157
   Progress: 4000/7157
   Progress: 2000/7157
   Progress: 3000/7157
   Progress: 4000/7157
   Progress: 5000/7157
   Progress: 6000/7157
   Progress: 7000/7157
   Progress: 5000/7157
   Progress: 6000/7157
   Progress: 7000/7157
✅ Database features created: (7157, 36)
📊 Features include: user demographics, game properties, category preferences, activity metrics

📊 Database Feature Summary:
   Total features: 36
   Key category features: category_preference_score, is_top_preferred_category
   Database-specific features: user_avg_score, games_played, game_average_rating
   Target variable: rating (1-5 rating scale)
✅ Database features created: (7157, 36)
📊 Features include: user demographics, game pr

# Train Database-Compatible Model

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

def train_database_model(X, y):
    """Train model optimized for database schema"""
    print(" TRAINING DATABASE-COMPATIBLE MODEL")
    print("=" * 60)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Optimize Gradient Boosting (best performer from previous tests)
    param_grid = {
        'n_estimators': [200, 300],
        'max_depth': [6, 8],
        'learning_rate': [0.05, 0.1],
        'subsample': [0.8, 0.9]
    }

    print(" Optimizing GradientBoosting for database schema...")

    gb_model = GradientBoostingRegressor(random_state=42)
    grid_search = GridSearchCV(
        gb_model,
        param_grid,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        verbose=1
    )

    grid_search.fit(X_train_scaled, y_train)
    best_model = grid_search.best_estimator_

    # Evaluate model
    y_pred_train = best_model.predict(X_train_scaled)
    y_pred_test = best_model.predict(X_test_scaled)

    train_mse = mean_squared_error(y_train, y_pred_train)
    test_mse = mean_squared_error(y_test, y_pred_test)
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)

    # Cross-validation
    cv_scores = cross_val_score(best_model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_mse = -cv_scores.mean()

    print(f" DATABASE MODEL RESULTS:")
    print(f"   Best Parameters: {grid_search.best_params_}")
    print(f"   Train MSE: {train_mse:.4f} | Test MSE: {test_mse:.4f}")
    print(f"   Train MAE: {train_mae:.4f} | Test MAE: {test_mae:.4f}")
    print(f"   CV MSE: {cv_mse:.4f}")

    # Feature importance analysis
    if hasattr(best_model, 'feature_importances_'):
        feature_importance = pd.DataFrame({
            'feature': X.columns,
            'importance': best_model.feature_importances_
        }).sort_values('importance', ascending=False)

        print(f"\n🔍 TOP 10 DATABASE FEATURES:")
        for _, row in feature_importance.head(10).iterrows():
            print(f"  {row['feature']}: {row['importance']:.4f}")

        # Analyze database-specific feature importance
        db_features = feature_importance[
            feature_importance['feature'].str.contains('score|games_played|average_rating|category|pref', case=False)
        ]

        print(f"\n📊 DATABASE-SPECIFIC FEATURE IMPORTANCE:")
        db_importance_sum = db_features['importance'].sum()
        print(f"Total database feature importance: {db_importance_sum:.4f}")

    return best_model, scaler, {
        'best_params': grid_search.best_params_,
        'train_mse': train_mse,
        'test_mse': test_mse,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'cv_mse': cv_mse
    }

# Train the database-compatible model
db_model, db_scaler, db_results = train_database_model(db_features_df, db_target)

print(f"\n Database Model Performance:")
print(f"   MSE: {db_results['test_mse']:.4f}")
print(f"   MAE: {db_results['test_mae']:.4f}")
print(f"   Ready for web application integration!")

🚀 TRAINING DATABASE-COMPATIBLE MODEL
🔧 Optimizing GradientBoosting for database schema...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
✅ DATABASE MODEL RESULTS:
   Best Parameters: {'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 200, 'subsample': 0.9}
   Train MSE: 0.1617 | Test MSE: 0.3254
   Train MAE: 0.3124 | Test MAE: 0.4372
   CV MSE: 0.3399

🔍 TOP 10 DATABASE FEATURES:
  category_preference_score: 0.6447
  preference_quality_interaction: 0.0622
  age_quality_interaction: 0.0387
  game_average_rating: 0.0275
  score_preference_interaction: 0.0230
  user_age: 0.0179
  user_pref_Adventure: 0.0145
  user_avg_score: 0.0131
  user_max_score: 0.0129
  user_pref_Action: 0.0114

📊 DATABASE-SPECIFIC FEATURE IMPORTANCE:
Total database feature importance: 0.9266

🎯 Database Model Performance:
   MSE: 0.3254
   MAE: 0.4372
   Ready for web application integration!
✅ DATABASE MODEL RESULTS:
   Best Parameters: {'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 200

# Database-Compatible Recommendation System

In [ ]:
class DatabaseGameRecommendationSystem:
    """Recommendation system designed for your database schema"""

    def __init__(self, model, scaler, feature_columns, user_preferences):
        self.model = model
        self.scaler = scaler
        self.feature_columns = feature_columns
        self.user_preferences = user_preferences

    def get_user_profile_from_db(self, user_data, user_ratings=None, user_games=None):
        """Extract user profile from database tables"""
        # Calculate age from birthDate
        birth_date = user_data['birthDate']
        if isinstance(birth_date, str):
            birth_date = datetime.strptime(birth_date, '%Y-%m-%d').date()

        age = calculate_user_age(birth_date)

        # Get gaming activity from scores
        if user_games and len(user_games) > 0:
            avg_score = np.mean([g['score'] for g in user_games])
            games_played = len(user_games)
            max_score = max([g['score'] for g in user_games])
        else:
            avg_score = 500  # Default
            games_played = 1
            max_score = 500

        # Get category preferences from ratings
        if user_ratings and len(user_ratings) > 0:
            category_prefs = self.analyze_user_ratings(user_ratings)
        else:
            category_prefs = {}

        return {
            'id': user_data['id'],
            'age': age,
            'gender': user_data['gender'],
            'avg_score': avg_score,
            'games_played': games_played,
            'max_score': max_score,
            'category_preferences': category_prefs
        }

    def analyze_user_ratings(self, user_ratings):
        """Analyze user's category preferences from ratings"""
        # Group ratings by category and calculate average rating
        category_ratings = {}

        for rating in user_ratings:
            # You would join with game and category tables here
            # For now, assuming category is provided with rating
            category = rating.get('category_name', 'Action')  # Default fallback
            rating_value = rating['rating']

            if category not in category_ratings:
                category_ratings[category] = []
            category_ratings[category].append(rating_value)

        # Calculate preferences
        preferences = {}
        for category, ratings in category_ratings.items():
            if len(ratings) >= 2:  # Need at least 2 ratings for reliability
                avg_rating = np.mean(ratings)
                count = len(ratings)
                # Weight by frequency slightly
                weighted_score = avg_rating * (1 + np.log(count) * 0.1)
                preferences[category] = weighted_score
            elif len(ratings) == 1:
                preferences[category] = ratings[0] * 0.8  # Reduce confidence

        return preferences

    def predict_user_rating(self, user_profile, game_data, category_data):
        """Predict how much a user would rate a specific game"""
        # Get category name
        category_name = category_data['name']

        # Create feature vector
        features = {
            'user_age': user_profile['age'],
            'user_gender_MALE': 1 if user_profile['gender'] == 'MALE' else 0,
            'user_gender_FEMALE': 1 if user_profile['gender'] == 'FEMALE' else 0,

            'user_avg_score': user_profile['avg_score'],
            'user_games_played': user_profile['games_played'],
            'user_max_score': user_profile['max_score'],

            'game_average_rating': game_data['averageRating'],
            'game_min_age': game_data['minAge'],
            'game_target_gender_MALE': 1 if game_data['targetGender'] == 'MALE' else 0,
            'game_target_gender_FEMALE': 1 if game_data['targetGender'] == 'FEMALE' else 0,

            'age_appropriate': 1 if user_profile['age'] >= game_data['minAge'] else 0,
            'gender_target_match': 1 if user_profile['gender'] == game_data['targetGender'] else 0,
        }

        # Category features
        user_prefs = user_profile['category_preferences']
        for category in CATEGORIES:
            features[f'game_category_{category}'] = 1 if category_name == category else 0

            if category in user_prefs:
                features[f'user_pref_{category}'] = user_prefs[category] / 5.0
            else:
                features[f'user_pref_{category}'] = 0.6  # Neutral

        # Key preference features
        features['category_preference_score'] = user_prefs.get(category_name, 3.0) / 5.0

        if user_prefs:
            top_category = max(user_prefs.items(), key=lambda x: x[1])[0]
            features['is_top_preferred_category'] = 1 if category_name == top_category else 0
        else:
            features['is_top_preferred_category'] = 0

        # Interaction features
        features.update({
            'age_quality_interaction': user_profile['age'] * game_data['averageRating'],
            'preference_quality_interaction': features['category_preference_score'] * game_data['averageRating'],
            'activity_preference_interaction': user_profile['games_played'] * features['category_preference_score'],
            'score_preference_interaction': (user_profile['avg_score'] / 1000) * features['category_preference_score']
        })

        # Convert to array and predict
        feature_vector = np.array([features.get(col, 0) for col in self.feature_columns]).reshape(1, -1)
        feature_vector_scaled = self.scaler.transform(feature_vector)
        predicted_rating = self.model.predict(feature_vector_scaled)[0]

        return max(1.0, min(5.0, predicted_rating))

    def recommend_games_from_db(self, user_data, all_games, all_categories, user_ratings=None, user_games=None, n_recommendations=5):
        """Generate recommendations using database tables"""
        print(f"\n DATABASE RECOMMENDATIONS")
        print(f"User: {user_data['username']} (Age: {calculate_user_age(user_data['birthDate'])}, Gender: {user_data['gender']})")
        print("=" * 60)

        # Get user profile
        user_profile = self.get_user_profile_from_db(user_data, user_ratings, user_games)

        # Show user preferences if available
        if user_profile['category_preferences']:
            sorted_prefs = sorted(user_profile['category_preferences'].items(), key=lambda x: x[1], reverse=True)
            print(f"User preferences: {[(cat, f'{score:.2f}') for cat, score in sorted_prefs[:3]]}")

        # Filter age-appropriate games
        eligible_games = [game for game in all_games if game['minAge'] <= user_profile['age']]

        # Filter out already rated games
        if user_ratings:
            rated_game_ids = {rating['gameId'] for rating in user_ratings}
            eligible_games = [game for game in eligible_games if game['id'] not in rated_game_ids]

        print(f"Analyzing {len(eligible_games)} eligible games...")

        # Generate predictions
        recommendations = []
        categories_dict = {cat['id']: cat for cat in all_categories}

        for game in eligible_games:
            category_data = categories_dict[game['categoryId']]
            predicted_rating = self.predict_user_rating(user_profile, game, category_data)

            recommendations.append({
                'game_id': game['id'],
                'game_name': next((item['name'] for item in content_items_df.to_dict('records') 
                                  if item['id'] == game['id']), "Unknown"),
                'game_image': next((item['imageUrl'] for item in content_items_df.to_dict('records') 
                                   if item['id'] == game['id']), None),
                'category_name': category_data['name'],
                'min_age': game['minAge'],
                'target_gender': game['targetGender'],
                'average_rating': game['averageRating'],
                'predicted_rating': predicted_rating,
                'is_preferred_category': category_data['name'] in user_profile['category_preferences']
            })

        # Sort by predicted rating
        recommendations.sort(key=lambda x: x['predicted_rating'], reverse=True)
        top_recommendations = recommendations[:n_recommendations]

        print(f"\nTop {n_recommendations} Recommendations:")
        print("-" * 60)

        for i, rec in enumerate(top_recommendations, 1):
            pref_mark = "⭐" if rec['is_preferred_category'] else ""
            print(f"{i}. {rec['game_name']} {pref_mark}")
            print(f"   Category: {rec['category_name']} | Target: {rec['target_gender']}")
            print(f"   Game Rating: {rec['average_rating']}/5.0 | Predicted: {rec['predicted_rating']:.2f}/5.0")
            print(f"   Min Age: {rec['min_age']}+ | Image: {rec['game_image']}")
            print()

        return top_recommendations

# Initialize database recommendation system
db_recommender = DatabaseGameRecommendationSystem(
    db_model, db_scaler, db_features_df.columns, db_user_preferences
)

print(f" Database Recommendation System Ready!")
print(f" Optimized for your web application database schema")
print(f" Performance: MSE={db_results['test_mse']:.4f}, MAE={db_results['test_mae']:.4f}")

✅ Database Recommendation System Ready!
🎯 Optimized for your web application database schema
📊 Performance: MSE=0.3254, MAE=0.4372


# Test Database Integration with Sample Data

In [ ]:
def test_database_integration():
    """Test the recommendation system with database-like data"""
    print(" TESTING DATABASE INTEGRATION")
    print("=" * 60)

    # Sample user data (as it would come from your database)
    sample_user = {
        'id': str(uuid.uuid4()),
        'name': 'John Doe',
        'username': 'johndoe',
        'email': 'john@example.com',
        'gender': 'MALE',
        'birthDate': datetime(1995, 5, 15).date(),  # 28 years old
        'role': 'USER'
    }

    # Sample user ratings (from GameRating table)
    sample_ratings = [
        {'id': str(uuid.uuid4()), 'userId': 1, 'gameId': games_df.iloc[0]['id'], 'rating': 5, 'category_name': 'Action'},
        {'id': str(uuid.uuid4()), 'userId': 1, 'gameId': games_df.iloc[1]['id'], 'rating': 4, 'category_name': 'Action'},
        {'id': str(uuid.uuid4()), 'userId': 1, 'gameId': games_df.iloc[2]['id'], 'rating': 2, 'category_name': 'Puzzle'},
        {'id': str(uuid.uuid4()), 'userId': 1, 'gameId': games_df.iloc[3]['id'], 'rating': 4, 'category_name': 'Strategy'},
        {'id': str(uuid.uuid4()), 'userId': 1, 'gameId': games_df.iloc[4]['id'], 'rating': 5, 'category_name': 'Shooter'},
    ]

    # Sample user games (from UserGame table)
    sample_user_games = [
        {'id': str(uuid.uuid4()), 'userId': 1, 'gameId': games_df.iloc[0]['id'], 'score': 850},
        {'id': str(uuid.uuid4()), 'userId': 1, 'gameId': games_df.iloc[1]['id'], 'score': 720},
        {'id': str(uuid.uuid4()), 'userId': 1, 'gameId': games_df.iloc[3]['id'], 'score': 950},
    ]

    # Convert our generated data to the format your database would provide
    sample_games = games_df.to_dict('records')[:50]  # First 50 games
    sample_categories = categories_df.to_dict('records')

    print(f"Testing with:")
    print(f"  User: {sample_user['username']} ({sample_user['gender']}, age {calculate_user_age(sample_user['birthDate'])})")
    print(f"  Ratings: {len(sample_ratings)} games rated")
    print(f"  Games: {len(sample_user_games)} games played")
    print(f"  Available games: {len(sample_games)}")

    # Generate recommendations
    recommendations = db_recommender.recommend_games_from_db(
        user_data=sample_user,
        all_games=sample_games,
        all_categories=sample_categories,
        user_ratings=sample_ratings,
        user_games=sample_user_games,
        n_recommendations=5
    )

    print(f"\n RECOMMENDATION ANALYSIS:")
    if recommendations:
        categories_recommended = [r['category_name'] for r in recommendations]
        avg_predicted_rating = np.mean([r['predicted_rating'] for r in recommendations])
        preferred_category_count = sum(1 for r in recommendations if r['is_preferred_category'])

        print(f"  Categories recommended: {categories_recommended}")
        print(f"  Average predicted rating: {avg_predicted_rating:.2f}")
        print(f"  Preferred categories: {preferred_category_count}/{len(recommendations)}")

        # Show how this matches user's actual preferences
        user_actual_prefs = {}
        for rating in sample_ratings:
            cat = rating['category_name']
            if cat not in user_actual_prefs:
                user_actual_prefs[cat] = []
            user_actual_prefs[cat].append(rating['rating'])

        user_avg_by_category = {cat: np.mean(ratings) for cat, ratings in user_actual_prefs.items()}
        sorted_user_prefs = sorted(user_avg_by_category.items(), key=lambda x: x[1], reverse=True)

        print(f"  User's actual preferences: {sorted_user_prefs}")

        # Check if recommendations align with preferences
        top_user_categories = [cat for cat, _ in sorted_user_prefs[:2]]
        recommended_categories = set(categories_recommended)
        alignment = len(set(top_user_categories) & recommended_categories)

        print(f"  Preference alignment: {alignment}/{len(top_user_categories)} top categories matched")

# Test the database integration
test_database_integration()

# Create API-like function for easy integration
def get_recommendations_for_user(user_id, n_recommendations=5):
    """
    API function for your web application

    Args:
        user_id (str): User ID from your database
        n_recommendations (int): Number of games to recommend

    Returns:
        list: Recommended games with all necessary information
    """
    # In your actual implementation, you would:
    # 1. Query your database for user data
    # 2. Query for user's ratings and games
    # 3. Query for all available games and categories
    # 4. Call the recommendation system

    # For demo, using our generated data
    user_data_row = users_df[users_df['id'] == user_id].iloc[0] if user_id in users_df['id'].values else users_df.iloc[0]
    
    # Create user data in database format
    user_data = {
        'id': user_id,
        'username': f'user{1}',
        'gender': user_data_row['gender'],
        'birthDate': user_data_row['birthDate']
    }
    
    # Get ratings and games for this user
    user_ratings_data = game_ratings_df[game_ratings_df['userId'] == 1].to_dict('records')  # Using userId 1 for demo
    user_games_data = user_games_df[user_games_df['userId'] == 1].to_dict('records')  # Using userId 1 for demo

    # Add category names to ratings (in real app, this would be a JOIN)
    for rating in user_ratings_data:
        game_data = games_df[games_df['id'] == rating['gameId']].iloc[0]
        category_data = categories_df[categories_df['id'] == game_data['categoryId']].iloc[0]
        rating['category_name'] = category_data['name']

    all_games = games_df.to_dict('records')
    all_categories = categories_df.to_dict('records')

    return db_recommender.recommend_games_from_db(
        user_data=user_data,
        all_games=all_games,
        all_categories=all_categories,
        user_ratings=user_ratings_data,
        user_games=user_games_data,
        n_recommendations=n_recommendations
    )

print(f"\n API FUNCTION READY:")
print(f"  Use: get_recommendations_for_user(user_id, n_recommendations)")
print(f"  Returns: List of recommended games with all database fields")

# Demo the API function
print(f"\n API DEMO:")
try:
    # Use first user ID for demo
    sample_user_id = users_df.iloc[0]['id']
    sample_recommendations = get_recommendations_for_user(sample_user_id, 3)
    print(" API function working correctly!")
except Exception as e:
    print(f" Error in API function: {e}")

🧪 TESTING DATABASE INTEGRATION
Testing with:
  User: johndoe (MALE, age 30)
  Ratings: 5 games rated
  Games: 3 games played
  Available games: 50

🎯 DATABASE RECOMMENDATIONS
User: johndoe (Age: 30, Gender: MALE)
User preferences: [('Action', '4.81'), ('Shooter', '4.00'), ('Strategy', '3.20')]
Analyzing 45 eligible games...

Top 5 Recommendations:
------------------------------------------------------------
1. Game 33 ⭐
   Category: Action | Target: MALE
   Game Rating: 4.87/5.0 | Predicted: 4.78/5.0
   Min Age: 7+ | Image: game33.jpg

2. Game 18 ⭐
   Category: Action | Target: MALE
   Game Rating: 4.57/5.0 | Predicted: 4.50/5.0
   Min Age: 7+ | Image: game18.jpg

3. Game 32 ⭐
   Category: Action | Target: MALE
   Game Rating: 4.11/5.0 | Predicted: 4.47/5.0
   Min Age: 3+ | Image: game32.jpg

4. Game 49 ⭐
   Category: Action | Target: MALE
   Game Rating: 4.18/5.0 | Predicted: 4.47/5.0
   Min Age: 3+ | Image: game49.jpg

5. Game 45 
   Category: RPG | Target: MALE
   Game Rating: 4.49/

# Save Database-Compatible System

In [ ]:
import pickle
import json
from datetime import datetime

def save_database_system():
    print(" SAVING DATABASE-COMPATIBLE RECOMMENDATION SYSTEM")
    print("=" * 70)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Save database tables
    print("📊 Saving Database Tables:")
    categories_df.to_csv(f'db_categories_table_{timestamp}.csv', index=False)
    users_df_simple.to_csv(f'db_users_table_{timestamp}.csv', index=False)
    content_items_df.to_csv(f'db_content_items_table_{timestamp}.csv', index=False)
    games_df.to_csv(f'db_games_table_{timestamp}.csv', index=False)
    game_ratings_df.to_csv(f'db_game_ratings_table_{timestamp}.csv', index=False)
    user_games_df.to_csv(f'db_user_games_table_{timestamp}.csv', index=False)

    print(" db_categories_table.csv (Category table)")
    print(" db_users_table.csv (Users table)")
    print(" db_content_items_table.csv (ContentItem table)")
    print(" db_games_table.csv (Game table)")
    print(" db_game_ratings_table.csv (GameRating table)")
    print(" db_user_games_table.csv (UserGame table)")

    # Save model artifacts
    print("\n Saving Database Model:")
    with open(f'db_recommendation_model_{timestamp}.pkl', 'wb') as f:
        pickle.dump(db_model, f)

    with open(f'db_model_scaler_{timestamp}.pkl', 'wb') as f:
        pickle.dump(db_scaler, f)

    with open(f'db_feature_columns_{timestamp}.pkl', 'wb') as f:
        pickle.dump(list(db_features_df.columns), f)

    print(f"   db_recommendation_model_{timestamp}.pkl")
    print(f"   db_model_scaler_{timestamp}.pkl")
    print(f"   db_feature_columns_{timestamp}.pkl")

    # Save database integration code
    integration_code = '''
# Database Integration Code for Your Web Application

import pickle
import numpy as np
import pandas as pd
from datetime import datetime

# Load the trained model
def load_recommendation_system():
    with open('db_recommendation_model.pkl', 'rb') as f:
        model = pickle.load(f)
    with open('db_model_scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)
    with open('db_feature_columns.pkl', 'rb') as f:
        feature_columns = pickle.load(f)

    return model, scaler, feature_columns

# Your web application integration function
def get_game_recommendations(user_id, connection, n_recommendations=5):
    """
    Get recommendations for a user from your database

    Args:
        user_id (str): User ID from your database
        connection: Your database connection
        n_recommendations (int): Number of games to recommend

    Returns:
        list: Recommended games with all information
    """

    # Load model
    model, scaler, feature_columns = load_recommendation_system()

    # 1. Get user data
    user_query = "SELECT * FROM Users WHERE id = ?"
    user_data = execute_query(connection, user_query, [user_id])

    # 2. Get user's ratings
    ratings_query = """
        SELECT gr.*, c.name as category_name
        FROM GameRating gr
        JOIN Game g ON gr.gameId = g.id
        JOIN Category c ON g.categoryId = c.id
        WHERE gr.userId = ?
    """
    user_ratings = execute_query(connection, ratings_query, [user_id])

    # 3. Get user's games
    games_query = "SELECT * FROM UserGame WHERE userId = ?"
    user_games = execute_query(connection, games_query, [user_id])

    # 4. Get all available games
    all_games_query = """
        SELECT g.*, c.name as category_name, ci.name as game_name, ci.imageUrl
        FROM Game g
        JOIN Category c ON g.categoryId = c.id
        JOIN ContentItem ci ON g.id = ci.id
    """
    all_games = execute_query(connection, all_games_query)

    # 5. Get all categories
    categories_query = "SELECT * FROM Category"
    all_categories = execute_query(connection, categories_query)

    # 6. Generate recommendations using the model
    return generate_recommendations(
        user_data[0], user_ratings, user_games, all_games, all_categories,
        model, scaler, feature_columns, n_recommendations
    )

# Helper function to calculate user age
def calculate_user_age(birth_date):
    today = datetime.now().date()
    return today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))

# Main recommendation function
def generate_recommendations(user_data, user_ratings, user_games, all_games, all_categories, model, scaler, feature_columns, n_recommendations):
    # Calculate user profile
    user_age = calculate_user_age(user_data['birthDate'])

    # Analyze user preferences from ratings
    user_preferences = analyze_user_category_preferences(user_ratings)

    # Calculate gaming activity from games
    if user_games:
        avg_score = np.mean([g['score'] for g in user_games])
        games_played = len(user_games)
        max_score = max([g['score'] for g in user_games])
    else:
        avg_score = 500
        games_played = 1
        max_score = 500

    # Filter age-appropriate games user hasn't rated
    rated_game_ids = {rating['gameId'] for rating in user_ratings}
    eligible_games = [
        game for game in all_games
        if game['minAge'] <= user_age and game['id'] not in rated_game_ids
    ]

    recommendations = []

    for game in eligible_games:
        # Create feature vector for this user-game combination
        features = create_feature_vector(
            user_data, user_age, user_preferences, avg_score, games_played, max_score, game
        )

        # Predict rating
        feature_array = np.array([features.get(col, 0) for col in feature_columns]).reshape(1, -1)
        feature_scaled = scaler.transform(feature_array)
        predicted_rating = model.predict(feature_scaled)[0]
        predicted_rating = max(1.0, min(5.0, predicted_rating))

        recommendations.append({
            'game_id': game['id'],
            'game_name': game['game_name'],
            'game_image': game['imageUrl'],
            'category_name': game['category_name'],
            'min_age': game['minAge'],
            'target_gender': game['targetGender'],
            'average_rating': game['averageRating'],
            'predicted_rating': predicted_rating
        })

    # Sort by predicted rating and return top N
    recommendations.sort(key=lambda x: x['predicted_rating'], reverse=True)
    return recommendations[:n_recommendations]

# Feature creation function (implement based on your model)
def create_feature_vector(user_data, user_age, user_preferences, avg_score, games_played, max_score, game):
    # This should match the feature engineering in your trained model
    # Implementation details provided in the full system
    pass

def analyze_user_category_preferences(user_ratings):
    # Analyze user's category preferences from their ratings
    # Implementation details provided in the full system
    pass
'''

    with open(f'database_integration_code_{timestamp}.py', 'w') as f:
        f.write(integration_code)

    print(f"\n Database Integration:")
    print(f"   database_integration_code_{timestamp}.py")

    # Save metadata
    db_metadata = {
        'timestamp': timestamp,
        'system_type': 'Database-Compatible Game Recommendation System',
        'model_performance': {
            'test_mse': db_results['test_mse'],
            'test_mae': db_results['test_mae'],
            'cv_mse': db_results['cv_mse']
        },
        'database_schema': {
            'tables_used': ['Users', 'Game', 'GameRating', 'UserGame', 'Category', 'ContentItem'],
            'key_fields': {
                'Users': ['id', 'birthDate', 'gender'],
                'Game': ['id', 'categoryId', 'minAge', 'targetGender', 'averageRating'],
                'GameRating': ['userId', 'gameId', 'rating'],
                'UserGame': ['userId', 'gameId', 'score'],
                'Category': ['id', 'name'],
                'ContentItem': ['id', 'name', 'imageUrl']
            }
        },
        'features': {
            'total_features': len(db_features_df.columns),
            'user_features': ['age', 'gender', 'gaming_activity'],
            'game_features': ['average_rating', 'min_age', 'target_gender', 'category'],
            'key_features': ['category_preference_score', 'is_top_preferred_category']
        },
        'integration': {
            'input_format': 'Database tables (Users, Game, GameRating, UserGame, Category)',
            'output_format': 'List of recommended games with metadata',
            'api_function': 'get_game_recommendations(user_id, connection, n_recommendations)'
        }
    }

    with open(f'database_system_metadata_{timestamp}.json', 'w') as f:
        json.dump(db_metadata, f, indent=2, default=str)

    # Save current versions for easy access
    print(f"\n Current Version Files:")
    categories_df.to_csv('current_db_categories.csv', index=False)
    users_df_simple.to_csv('current_db_users.csv', index=False)
    content_items_df.to_csv('current_db_content_items.csv', index=False)
    games_df.to_csv('current_db_games.csv', index=False)
    game_ratings_df.to_csv('current_db_game_ratings.csv', index=False)
    user_games_df.to_csv('current_db_user_games.csv', index=False)

    with open('db_recommendation_model.pkl', 'wb') as f:
        pickle.dump(db_model, f)
    with open('db_model_scaler.pkl', 'wb') as f:
        pickle.dump(db_scaler, f)
    with open('db_feature_columns.pkl', 'wb') as f:
        pickle.dump(list(db_features_df.columns), f)
    with open('database_integration_guide.py', 'w') as f:
        f.write(integration_code)

    print("   current_db_categories.csv")
    print("   current_db_users.csv")
    print("   current_db_content_items.csv")
    print("   current_db_games.csv")
    print("   current_db_game_ratings.csv")
    print("   current_db_user_games.csv")
    print("   db_recommendation_model.pkl")
    print("   db_model_scaler.pkl")
    print("   database_integration_guide.py")

    return db_metadata

# Save the database system
db_metadata = save_database_system()

print(f"\n🎉 DATABASE SYSTEM SAVED SUCCESSFULLY!")
print("=" * 60)
print(f" Model Performance: MSE={db_results['test_mse']:.4f}, MAE={db_results['test_mae']:.4f}")
print(f" Database Tables: {len(categories_df)} categories, {len(users_df_simple)} users, {len(games_df)} games")
print(f" Training Data: {len(game_ratings_df)} ratings, {len(user_games_df)} scores")
print(f" Ready for web application integration!")

print(f"\n📋 INTEGRATION CHECKLIST:")
print(" Model trained on your exact database schema")
print(" Features match your table fields (Users, Game, GameRating, UserGame, Category)")
print(" Age calculated from birthDate")
print(" Gender mapping (MALE/FEMALE)")
print(" Category preferences from user ratings")
print(" Gaming activity from UserGame table")
print(" Integration code provided")
print(" API function ready: get_game_recommendations(user_id, connection)")

print(f"\n Your recommendation system is now optimized for your database schema!")

💾 SAVING DATABASE-COMPATIBLE RECOMMENDATION SYSTEM
📊 Saving Database Tables:
  ✅ db_categories_table.csv (Category table)
  ✅ db_users_table.csv (Users table)
  ✅ db_content_items_table.csv (ContentItem table)
  ✅ db_games_table.csv (Game table)
  ✅ db_game_ratings_table.csv (GameRating table)
  ✅ db_user_games_table.csv (UserGame table)

🤖 Saving Database Model:
  ✅ db_recommendation_model_20250601_014804.pkl
  ✅ db_model_scaler_20250601_014804.pkl
  ✅ db_feature_columns_20250601_014804.pkl

💻 Database Integration:
  ✅ database_integration_code_20250601_014804.py

📁 Current Version Files:
  ✅ current_db_categories.csv
  ✅ current_db_users.csv
  ✅ current_db_content_items.csv
  ✅ current_db_games.csv
  ✅ current_db_game_ratings.csv
  ✅ current_db_user_games.csv
  ✅ db_recommendation_model.pkl
  ✅ db_model_scaler.pkl
  ✅ database_integration_guide.py

🎉 DATABASE SYSTEM SAVED SUCCESSFULLY!
🏆 Model Performance: MSE=0.3254, MAE=0.4372
🗄️ Database Tables: 9 categories, 500 users, 1000 games